In [0]:
from fastai.text import * 

In [0]:

from google.colab import files 
uploaded = files.upload() 


Saving test.csv to test.csv
Saving train.csv to train.csv


In [0]:
import pandas as pd

# df_train = pd.read_csv("train.csv", names=["cat", "text"])
# df_test = pd.read_csv("test.csv", names=["cat", "text"])

train = pd.read_csv("train.csv", names=["cat", "useless", "text"])
test = pd.read_csv("test.csv", names=["cat", "useless", "text"])

df_train = train[["cat", "text"]]
df_test = test[["cat", "text"]]

!ls

In [0]:
df_train.head()

In [0]:
set(df_train["cat"])

In [11]:
from sklearn.model_selection import train_test_split

df_trn, df_val = train_test_split(df_train, stratify = df_train['cat'], test_size = 0.33)

df_trn.shape, df_val.shape

((80400, 2), (39600, 2))

In [12]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "", vocab=data_lm.train_ds.vocab, bs=32)


# # Language model data
# data_lm = TextLMDataBunch.from_df(train_df = df_train, path = "")

# # Classifier model data
# data_clas = TextClasDataBunch.from_df(train_df = df_train, path = "", vocab=data_lm.train_ds.vocab, bs=32)


In [13]:
data_clas.show_batch()


text,target
"xxbos 4 xxmaj miami at xxup n.c. xxmaj state < xxunk p.m. , xxup espn < / em><br > xxmaj think the xxmaj wolfpack is kicking itself for that loss two weeks ago at xxmaj north xxmaj carolina ? xxmaj you bet . xxmaj had xxup n.c. xxmaj state ( 4 - 2 , 3 - 1 xxup acc ) won that one , this would be for sole possession",2
xxbos \ \ xxmaj today xxmaj brad gave me a quick demo of xxmaj google xxmaj desktop . \ \ xxmaj its great that xxmaj google keeps innovating ( and more power to them ) but to be honest \ i was a bit xxunk . \ \ caption : xxmaj google xxmaj desktop xxmaj homepage . \ \ xxmaj it should be possible to quickly put together a xxmaj,4
"xxbos < em > xxmaj dean xxmaj cain has spent much of his life in a uniform . xxmaj he 's done time as an all - xxmaj american safety at xxmaj princeton ( where he established a since - broken single - season xxmaj division i - xxup aa record for interceptions ) , an undrafted free agent with the xxmaj buffalo xxmaj bills ( before a preseason knee",2
"xxbos xxup san xxup francisco - xxmaj as xxmaj sun xxmaj microsystems xxmaj inc. and xxmaj microsoft xxmaj corp. ready an early xxmaj december unveiling of the first step in their highly publicized technology collaboration , industry observers are questioning whether the former foes will fail to address industry standards , particularly the work being done by the xxmaj sun - backed xxmaj liberty xxmaj alliance xxunk > xxup advertisement",4
"xxbos xxup moscow - xxmaj traces of explosives have been found in the wreckage of one of two airliners that crashed nearly simultaneously earlier this week , the xxmaj federal xxmaj security xxmaj service said xxmaj friday , a day after a top official acknowledged that terrorism was the most likely cause of the crashes . a duty officer at the agency , the main successor to the xxmaj soviet",1


In [0]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')
data_lm = load_data('./', 'data_lm_export.pkl')
data_clas = load_data('./', 'data_clas_export.pkl', bs=16)

In [15]:
# learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)


epoch,train_loss,valid_loss,accuracy,time
0,3.585423,3.345819,0.389579,02:28


In [16]:
# learn.lr_find()
# learn.recorder.plot(suggestion=True)
# min_grad_lr = learn.recorder.min_grad_lr

learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)


epoch,train_loss,valid_loss,accuracy,time
0,3.142862,3.106576,0.422262,02:50


In [17]:
# learn.fit_one_cycle(2, min_grad_lr)

learn.predict("This is a review about", n_words=10)


'This is a review about experts at the University of England whose problems'

In [0]:
# learn.unfreeze()
# learn.fit_one_cycle(2, 1e-3)

learn.save_encoder('ft_enc')


In [19]:
# learn.predict("How do", n_words=10)

learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')


RNNLearner(data=TextClasDataBunch;

Train: LabelList (80400 items)
x: TextList
xxbos a win today over xxmaj boston xxmaj college gives xxmaj west xxmaj virginia no worse than a share of the xxmaj big xxmaj east championship and a berth in either the xxmaj sugar xxmaj bowl or the xxmaj fiesta xxmaj bowl .,xxbos xxup baghdad : xxmaj at least 110 people have been killed across xxmaj iraq in a sharp escalation of violence that saw gun battles , car bombs and xxunk rock the capital .,xxbos ( xxmaj sports xxmaj network ) - xxmaj with losses in four of their last five games to non - playoff teams , the xxmaj chicago xxmaj cubs know they can ' t falter over their final four games of the season , or they will miss the playoffs .,xxbos xxmaj florida xxmaj supreme xxmaj court judges on xxmaj wednesday questioned attorneys for smokers and cigarette companies on whether a case involving thousands of smokers can properly be tried as a class action .,xxbos xxmaj the xxmaj united xxmaj states lobbied 

In [20]:
# learn.save_encoder('ft_enc')

data_clas.show_batch()


text,target
"xxbos xxmaj there 's troubling news ( xxup ft subscription xxunk , alternate copy here ) coming from xxmaj japan , where the xxmaj kyoto protocol on xxmaj greenhouse xxmaj emissions was born in 1997 . xxmaj it seems that the xxmaj japanese are n't going to be able to meet their emissions targets specified in the agreement in time . xxmaj indeed , unless they buy a "" large",4
"xxbos xxmaj the xxup u.s. xxmaj supreme xxmaj court declined xxmaj tuesday to hear appeals in two cases concerning the right of the entertainment industry to subpoena file - traders and whether telecommunication companies have to share their networks with startup xxunk > xxup advertisement < / p><p><img src=""http : / / ad.doubleclick.net / ad / idg.us.ifw.general / sbcspotrssfeed;sz=1x1;ord=200301151450 ? "" width=""1 "" height=""1 "" border=""0 "" / > <",4
"xxbos xxmaj the xxmaj motley xxmaj fool - xxmaj financial services firms like xxmaj citigroup ( xxup nyse : c - xxmaj news ) , xxmaj bank of xxmaj america ( xxup nyse : xxup bac - xxmaj news ) , and xxmaj morgan xxmaj stanley ( xxup nyse : xxup mwd - xxmaj news ) want to offer just about everything . xxmaj to do this requires huge investments",4
"xxbos xxmaj yesterday morning things seemed pretty dicey for the xxup nba 's xxmaj ron xxmaj artest , as chatter about "" the poster boy for bad behavior "" and talk of federal lawsuits , possible criminal charges and suspension appeals fluttered about the airwaves . < em > d < / xxunk > xxunk < / em > , people . xxmaj then there was the little matter of",2
"xxbos xxmaj college basketball : xxup men -- xxmaj boston xxmaj university at xxmaj harvard , 7 p.m. ; xxmaj wesleyan at xxmaj emmanuel , 7 p.m. ; xxmaj emerson at xxmaj babson , 7 p.m. ; xxmaj plymouth xxmaj st. at xxmaj tufts , 7 p.m. ; xxmaj newbury at xxmaj salem xxmaj st. , 7 p.m. ; xxmaj merrimack at xxmaj bentley , 7:30 p.m. ; xxmaj curry",2


In [21]:
# learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
# learn.load_encoder('ft_enc')

learn.fit_one_cycle(1, 1e-2)


epoch,train_loss,valid_loss,accuracy,time
0,0.399048,0.301736,0.895177,02:44


In [22]:
# learn.lr_find()
# learn.recorder.plot(suggestion=True)
# min_grad_lr = learn.recorder.min_grad_lr


learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))


epoch,train_loss,valid_loss,accuracy,time
0,0.307935,0.254577,0.913788,02:59


In [23]:
# learn.fit_one_cycle(2, min_grad_lr)


learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))


epoch,train_loss,valid_loss,accuracy,time
0,0.231002,0.227188,0.923662,05:06


In [24]:
# learn.recorder.plot_losses()

learn.predict("This was a great movie!")


(Category 3, tensor(2), tensor([0.1122, 0.2673, 0.3632, 0.2573]))

In [0]:
# learn.freeze_to(-2)
# learn.fit_one_cycle(4, slice(5e-3, 2e-3), moms=(0.8,0.7))


In [0]:
# learn.recorder.plot_losses()


In [0]:
# learn.unfreeze()
# learn.fit_one_cycle(4, slice(2e-3/100, 2e-3), moms=(0.8,0.7))


In [0]:
# preds,y,losses = learn.get_preds(with_loss=True)
# interp = ClassificationInterpretation(learn, preds, y, losses)
# interp.plot_confusion_matrix()


In [0]:
# interp.most_confused()


In [25]:
learn.data.add_test(df_test["text"])
prob_preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)


In [26]:
y = np.array(list(df_test["cat"]))
len(y)

7600

In [32]:
from sklearn.metrics import accuracy_score, f1_score
print(accuracy_score(y, np.argmax(prob_preds[0], axis=1)+1))
print(f1_score(y, np.argmax(prob_preds[0], axis=1)+1, average="macro"))

0.9219736842105263
0.9218615201647395


In [33]:
np.argmax(prob_preds[0], axis=1)+1

tensor([3, 4, 4,  ..., 2, 3, 3])

In [29]:
df_train.head()

,cat,text
0,3,"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Reuters - Private investment firm Carlyle Grou...
2,3,Reuters - Soaring crude prices plus worries\ab...
3,3,Reuters - Authorities have halted oil export\f...
4,3,"AFP - Tearaway world oil prices, toppling reco..."


In [30]:
set(df_train["cat"])

{1, 2, 3, 4}